Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
val NAME = ""
val COLLABORATORS = ""

---

# Lab1 Intro to Scala and Chisel
> Labs will be due each week before the homeworks. They are not intended take a significant amount of time but rather to provide examples/practice on specific and isolated features in the language. Labs are autograded so you can get quick feedback.

### Import the necessary Chisel dependencies. 
> There will be a cell like this in every lab. Make sure you run it before proceeding to bring the Chisel Library into the Jupyter Notebook scope!

In [ ]:
val path = System.getProperty("user.dir") + "/../resource/chisel_deps.sc"
interp.load.module(ammonite.ops.Path(java.nio.file.FileSystems.getDefault().getPath(path)))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chisel3.tester.RawTester.test

## Problem 1 (2 pts) - Scala conditionals
> Practice with Scala's if/else. Return `"heads"` if `flip` is `true`, else return `"tails"`

In [ ]:
def scalaCondPractice(flip: Boolean): String = {
    // YOUR CODE HERE
    ???
}

In [ ]:
assert(scalaCondPractice(true) == "heads")
assert(scalaCondPractice(false) == "tails")

## Problem 2 (3 pts) - Writing a Chisel Module's IO
> Fill in the given module's IO such that it takes two 4-bit `UInt`s as input and returns a 5-bit sum as output. 

In [ ]:
class AddTwo extends Module {
    // val io = ???
    // YOUR CODE HERE
    ???
    
    io.out := io.in1 +& io.in2
}

In [ ]:
def testAddTwo: Boolean = {
    test(new AddTwo) { c =>
        for (i <- 0 until 16) {
            for (j <- 0 until 16) {
                c.io.in1.poke(i.U)
                c.io.in2.poke(j.U)
                c.io.out.expect((i+j).U)
            }
        }
    }
    true
}

assert(testAddTwo)

## Problem 3 (3 pts) - Combinational Logic
> Assign the boolean expression: `(a AND b) OR (NOT c)` to the module's output. 

In [ ]:
class CombLogic extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val c   = Input(Bool())
        val out = Output(Bool())
    })
    
    // YOUR CODE HERE
    ???
    
    // We can print state like this everytime `step()` is called in our test
    printf(p"a: ${io.a}, b: ${io.b}, c: ${io.c}, out: ${io.out}\n")
}

## Problem 4 (4 pts) - Combinational Logic Test
> Write your own test that tests `CombLogic` exhaustively for all input values `a, b, and c`. The module should return `true` if and only if all calls to `dut.io.out.expect(...)` succeed.

In [ ]:
def testCombLogic: Boolean = {
    test(new CombLogic) { dut =>
        
        // YOUR CODE HERE
        ???
    }
    true
}

In [ ]:
assert(testCombLogic)


## Problem 5 (3 pts) - Scala Conditional in Chisel modules
> At hardware generation time we can use Scala conditionals to change which hardware is created within a module. In this exercise, if the `useAnd` argument is `true`, then output `a && b` otherwise output `a || b`. The module's verilog should contain only `AND` logic or `OR` logic, but not both.

In [ ]:
class AndOrGenerationTime(useAnd: Boolean) extends Module {
    val io = IO(new Bundle {
        val a   = Input(Bool())
        val b   = Input(Bool())
        val out = Output(Bool())
    })
    
    // YOUR CODE HERE
    ???
}

In [ ]:
def testAndOrGenerationTime(useAnd: Boolean): Boolean = {
    test(new AndOrGenerationTime(useAnd)) { dut =>
        for (a <- Seq(true, false)) {
            for (b <- Seq(true, false)) {
                dut.io.a.poke(a.B)
                dut.io.b.poke(b.B)
                if (useAnd) dut.io.out.expect((a && b).B)
                else        dut.io.out.expect((a || b).B)
            }
        }
    }
    true
}
assert(testAndOrGenerationTime(useAnd = true))
assert(testAndOrGenerationTime(useAnd = false))

## Problem 6 (3 pts) - Chisel Conditional in Chisel modules
> The generated hardware can use conditionals (i.e `Mux` or `when/elsewhen/otherwise`) to select which signals to assign. In this exercise, `useAnd` is an `Input` to the module. If `useAnd` is `true`, then set output `out` to the result of `a && b` otherwise to `a || b`. In this example both the logic for `a && b` and `a || b` hardware will be generated. You may use either a Chisel `Mux` or the Chisel `when` statements.

In [ ]:
class AndOrRunTime extends Module {
    val io = IO(new Bundle {
        val a      = Input(Bool())
        val b      = Input(Bool())
        val useAnd = Input(Bool())
        val out    = Output(Bool())
    })
    
    // YOUR CODE HERE
    ???
}

In [ ]:
def testAndOrRunTime: Boolean = {
    test(new AndOrRunTime) { dut =>
        for (a <- Seq(true, false)) {
            for (b <- Seq(true, false)) {
                for (useAnd <- Seq(true, false)) {
                    dut.io.a.poke(a.B)
                    dut.io.b.poke(b.B)
                    dut.io.useAnd.poke(useAnd.B)
                    if (useAnd) dut.io.out.expect((a && b).B)
                    else        dut.io.out.expect((a || b).B)
                }
            }
        }
    }
    true
}
assert(testAndOrRunTime)

## Problem 7 (2 pts) - Last connect semantics
> When assigning Chisel values, the last connection is the one that wins. In the module below the default output is `5.U` because `out` is assigned to `5.U` after `4.U`. Use a `when` statement to conditionally output `10.U` when the input `update` is set high or output its default when `update` is set low.

In [ ]:
class LastConnect extends Module {
    val io = IO(new Bundle {
        val update = Input(Bool())
        val out    = Output(UInt())
    })
    
    io.out := 4.U
    io.out := 5.U
    // YOUR CODE HERE
    ???
    
}

In [ ]:
def testLastConnect: Boolean = {
    test(new LastConnect) { dut =>
        dut.io.update.poke(true.B)
        dut.io.out.expect(10.U)
        
        dut.io.update.poke(false.B)
        dut.io.out.expect(5.U)
    }
    true
}
assert(testLastConnect)

## Problem 8 (5 pts) - Simple ReLU
> Let's put together some of these techniques to build a more complicated module. A ReLU or rectified linear unit is a function used in ML. (https://en.wikipedia.org/wiki/Rectifier_(neural_networks))

To combine everything we've learned so far we will slightly modify the function to saturate at a parameterized upper-bound of our choosing. The module will model this function: `f(x, upper_bound) = max(0, min(x, upperBound))`.

Here is an example where we parameterize `upperBound` to equal `3`, note the input `x` and output `y` will be of type SInt.

<img src="images/relu.png" style="width:60%;">

In [ ]:
class ReLU(upperBound: Int) extends Module {
    val io = IO(new Bundle {
        val x = Input(SInt(5.W))
        val y = Output(SInt(5.W))
    })
    // YOUR CODE HERE
    ???
}

## Problem 9 (5 pts) - ReLU Test
> Write your own test for `ReLU` that tests `x` at input values `-1, 0, 1, 15`. The test (`testReLU`) is parameterized by `upperBound`, and you can assume `upperBound` is non-negative. The module should return `true` if and only if all calls to `dut.io.y.expect(...)` succeed.

In [ ]:
def testReLU(upperBound: Int): Boolean = {
    require(upperBound >= 0)
    test(new ReLU(upperBound)) { dut =>
        
    // YOUR CODE HERE
    ???
    true
}

In [ ]:
for(upperBound <- 0 until 16) {
    println(s"Testing ReLu, upperBound=$upperBound")
    assert(testReLU(upperBound))
}